# Техника сглаживания для тензорных норм (20 pts)

Техника сглаживания оказала достаточно большое влияние на развитие методов восстановления тензоров (matrix completion problem является частным случаем). 

Рассмотрим задачу восстановления матрицы $Y$. Мы наблюдаем только небольшой набор элементов матрицы $Y$, множество индексов известных элементов обозначим через $E$. То есть мы знаем $Y_{ij}$ для всех $(i,j) \in E$, и не знаем $Y_{ij}$ для всех $(i,j) \notin E$.

Понятно, что без наличия какой-либо дополнительной информации о матрице $Y$ эта постановка является тривиальной: любая матрица $X$, для которой выполнено $X_{i,j} = Y_{i,j}, \quad (i,j) \in E$ является разумным ответом. Таких матриц бесконечно много. Соответственно задача восстановления матриц обычно рассматривается как частный случай задачи приближения матрицы, а критерий качества (целевая функция) говорит не столько о близости $X$ к $Y$, сколько о полезных свойствах матрицы $X$ (этим свойствам матрица $Y$ может и не обладать).

Одна из самых популярных постановок, с которой Вы наверняка сталкивались, например, строя рекомендательные системы, является задача о поиске наилучшего приближения заданной матрицы матрицей малого ранга:

$$
\begin{align*}
& \min_{X} rk(X) \\
& X_{i,j} = Y_{i,j}, \quad (i,j) \in E\\
\end{align*}
$$
Как известно, в общем случае эта задача является NP-трудной.

Для того, чтобы обойти это припятствие ранк матрицы аппроксимируется той или иной выпуклой функцией от матрицы $X$.

Опять же стандартным выбором является переход к постановке задачи с использованием 1-й нормы Шаттена (она же trace norm).

$\textbf{RegMC problem}$

$$
\begin{align*}
& \min_{X}\|X \|_* \\
& X_{i,j} = Y_{i,j}, \quad (i,j) \in E\\
\end{align*}
$$

Здесь $X_* = \sum \sigma_i(X)$. 


1. (5 pts) Найдите аналитическую запись для градиента сглаженной версии целевого функционала $\|X\|_*$ используя в качестве прокс-функции $d(Z) = \frac{1}{2}\|Z\|_F^2$.
2. (5 pts) Реализуйте быстрый градиентный спуск (лекция про сглаживание или статья Smooth minimization of non-smooth functions [Nesterov, 2005]) для  сглаженной версии задачи RegMC problem. В качестве прокс-функции для прямой задачи используйте $d(Z) = \frac{1}{2}\|Z\|_F^2$.
3. (5 pts) Реализуйте какую-либо версию метода проксимального градиентного спуска для решения RegMC problem. 
4. (5 pts) Протестируйте алгоритмы из п.2-3 на тестовых данных. Для построния тестовых данных возьмите произвольный датасет картинок. Для каждой картинки удалите случайный набор пикселей (от 10% до 90%). 
От Вас требуется сравнить скорость сходимости методов на тестовых данных. Предложите метод сравнения и обоснуйте свой выбор.





**1)**

$$f(X) = \|X \|_* = \max_{\|Z\|_2\leq 1}(tr(Z^TX)) = \max_{\|Z\|_2\leq 1}(\sum_j\sum_iZ_{ij}X_{ij})$$
Сглаженная функция:
$$ f_\mu(X) = \max_{\|Z\|_2\leq 1}(\sum_j\sum_iZ_{ij}X_{ij} - \frac{\mu}{2}\|Z\|_F^2) $$

Заметим, что:  
1)$\|Z\|_F^2 = \sum_j\sum_iZ_{ij}Z_{ij}$ 
 
2)$\|Z\|_2\leq 1 \iff Z^TZ \prec I \iff Z_{ii}^2\leq 1$

Тогда

$$L = \sum_j\sum_iZ_{ij}X_{ij} - \frac{\mu}{2}\sum_j\sum_iZ_{ij}Z_{ij} + \lambda_i\delta_{ij}(Z_{ii}^2-1)$$ где $\delta_{ij}$ - символ Кронекера
(будем далее понимать под $\langle X, Z\rangle$ - сумму $\sum_j\sum_iZ_{ij}X_{ij}$)
$$\frac{\partial L}{\partial Z_{ij}} = X_{ij}-\mu Z_{ij} + 2\lambda_i\delta_{ij}Z_{ii} = 0$$
$$Z_{ij} = \frac{X_{ij}}{\mu - 2\lambda_i\delta_{ij}}$$

для удобства заменим условие $\|Z\|_2\leq 1$ на $\|Z\|_F\leq 1$, так как $\|Z\|_2\leq \|Z\|_F$, мы усилим условие немного, но с этим будет проще работать, ведь тогда можно записать единую функцию Лагранжа: $$L = \langle X, Z\rangle - \frac{\mu}{2}\|Z\|_F^2+\lambda(\|Z\|_F^2 - 1)$$
откуда $Z^* = \frac{1}{\mu-2\lambda^*}X$, где $\lambda^* = 
\begin{cases}
0,\;\; \frac{1}{\mu^2}\|X\|_F^2\leq 1\\
\frac{\mu\pm\|X\|_F}{2},\;\; else
\end{cases}$

Тогда можем записать сглаженную версию функции $f$:

$$f_{\mu}(X) = \frac{\|X\|_F^2}{\mu-2\lambda^*}(1 - \frac{\mu}{2(\mu-2\lambda^*)}),\:\:\:\:\lambda^* = 
\begin{cases}
0,\;\; \frac{1}{\mu^2}\|X\|_F^2\leq 1\\
\frac{\mu\pm\|X\|_F}{2},\;\; else
\end{cases}$$

или же 
$$f_{\mu}(X) = 
\begin{cases}
\frac{1}{2\mu}\|X\|_F^2,\:\:\:\frac{1}{\mu^2}\|X\|_F^2\leq 1\\
\|X\|_F-\frac{\mu}{2},\:\:\:else
\end{cases}$$
(видим, что второй случай негладкий:(

Найдём градиент $f_{\mu}(X)$:

$$\nabla f_{\mu}(X) = 
\begin{cases}
\frac{1}{\mu}X,\:\:\:\frac{1}{\mu^2}\|X\|_F^2\leq 1\\
\frac{X}{\|X\|_F},\:\:\:else
\end{cases}$$

**новые рассчёты:**
Сглаженная функция:
$$ f_\mu(X) = \max_{\|Z\|_2\leq 1}(\sum_j\sum_iZ_{ij}X_{ij} - \frac{\mu}{2}\|Z\|_F^2) $$

Тогда из линейности $f_{\mu}$ по $X$ имеем $$\nabla f_{\mu} = arg\max_{\|Z\|_2\leq 1}(\sum_j\sum_iZ_{ij}X_{ij} - \frac{\mu}{2}\|Z\|_F^2)$$

Преобразуем:

$$arg\max_{\|Z\|_2\leq 1}(\sum_j\sum_iZ_{ij}X_{ij} - \frac{\mu}{2}\|Z\|_F^2) = arg\max_{\|Z\|_2\leq 1}(\langle Z, X\rangle - \frac{\mu}{2}\|Z\|_F^2) = arg\min_{\|Z\|_2\leq 1}(\frac{\mu}{2}\|Z\|_F^2 - \langle Z, X\rangle) = arg\min_{\|Z\|_2\leq 1}  (\langle Z, Z\rangle - \frac{2}{\mu}\langle Z, X\rangle) = arg\min_{\|Z\|_2\leq 1}  (\langle Z, Z\rangle - \frac{2}{\mu}\langle Z, X\rangle) + \frac{1}{\mu^2}\langle X, X\rangle) = arg\min_{\|Z\|_2\leq 1}  (\langle Z - \frac{1}{\mu}X, Z - \frac{1}{\mu}X\rangle) = arg\min_{\|Z\|_2\leq 1}\|Z - \frac{1}{\mu}X\|_F^2 $$

Пусть $\sigma_i$ - сингулярные числа матрицы $X$, тогда возьмём в качестве $Z$ матрицу: $Z = UE_ZV^T$, где $X=UEV^T$ и $E_z$ образована сингулярными числами $Z$ - $\sigma_z^i$, для которых $\sigma_z^i = \min(1, \frac{1}{\mu}\sigma_i)$, тогда такая матрица $Z$ удовлетворяет ограничениям по второй норме и получается, что каждое сингулярное число матрицы $Z - \frac{1}{\mu}X$ минимально, тогда это $Z$ и будет градиентом.
$$\nabla f_{\mu} = UE_ZV^T$$

**2)** Тут буду использовать собственно алгоритм Нестерова.

$Udiag(min(1,\frac{1}{\mu}\sigma_i))V^T = D(\frac{1}{\mu}X)$

Теперь ищем константы $\sigma$ и $L$ для алогритма

Так как $d(Z) = \frac{1}{2}||Z||_F^2$ и $d(Z) \geqslant \frac{\sigma}{2}||Z||_F^2$ то, очевидно $\sigma = 1$

имеем  $||\nabla f(x) - \nabla f(y)||_* \leqslant L||x-y||_F$

 Распишем разность градиентов, используя матрицу $D$:
  $\|\nabla f(x) - \nabla f(y)||_* = ||D(\frac{1}{\mu}(x-y))||_* = max_{||Z||_*=1}||D(\frac{1}{\mu}(x-y))^TZ||_* \leqslant ||D(\frac{1}{\mu}(x-y))||_*\leqslant\frac{1}{\mu}||x-y||_*$
  Отсюда $L = \frac{1}{\mu}$

  Теперь запишем алгоритм реализации быстрого градиентного спуска (для сглаженной задачи)

  1. Нужно вычислить $\nabla f(x_k)$ и $f(x_k)$.
  2. $y_k = argmin_{y\in Q}(<(y-x_k),\nabla f(x_k)> + \frac{1}{2\mu}||y-x||_F^2)$.
  
  Рассмотрим матричную компоненту $\alpha_{ij}$ матрицы $y$, которую мы можем менять (остальные компоненты не трогаем). Далее рассмотрим $<(y-x_k, \nabla f(x_k)>$. След содержит только числа вида $c_{kk}$, а $\alpha_{ij}$ входит только в $c_{jj} = \sum\limits_{m=1}^{n}\alpha_{mj}(\nabla f(x_k))$. Тогда получается, что при $\alpha_{ij}$ в итоговой сумме один из коэффициентов будет $(\nabla f(x_k))_{ij}$. Теперь рассмотрим $\frac{1}{2\mu}||y-x||_F^2$. Здесь аналогично, $\alpha_{ij}$ входит в $\frac{1}{2\mu}c_{jj} = \frac{1}{2\mu}\sum\limits_{m=1}^{n}\alpha_{mj}\alpha_{mj}$. Получается, что в итоговую сумму от этого слагаемого входит $(\nabla f(x_k))_{ij}\alpha_{ij}+\frac{1}{2\mu}\alpha_{ij}^2$. Минимум достигается при $\alpha_{ij} = -\mu(\nabla f(x_k))$.

  3. Теперь нужно вычислить $z_k = \underset{x\in Q}{argmin}(\frac{1}{\mu}||x||_F^2 + \sum\limits_{i=0}^{k}\frac{i+1}{2}[f(x_i) + <(x-x_i),\nabla f(x_i)>])$.

  Аналогично рассматриваем некоторую изменяемую компоненту матрицы $x - \alpha_{mj}$. Получаем, что $\alpha_{mj}$ входит в функционал в следующем виде: $\frac{1}{2\mu}\alpha_{mj}^2+\sum\limits_{i=0}^{k}\frac{i+1}{2}(\alpha_{mj} - (x_i)_{mj})(\nabla f(x_i))$. Минимум здесь достигается при $\alpha_{mj} = \mu\sum\limits_{i=0}^{k}\frac{i+1}{2}(\nabla f(x_i))_{mj}$.

  4. Получаем итерационное уравнение $x_{n+1} = \frac{n+1}{n+3}y_n + \frac{2}{n+3}z_n$
 

In [ ]:
import numpy as np
import copy

def exist(E, i, j):
  for e in E:
    if i == e[0] and j == e[1]:
      return True
  return False
def grad(x, mu):
  u, s, v = np.linalg.svd(x, full_matrices = False)
    
  s1 = np.minimum(np.ones_like(s), s/mu)
  G = np.dot(u*s1, v)
  return G
def ynew(y, x, notE, mu):
  size = len(notE)
  G = grad(x, mu)
  for i in range(size):
    y[notE[i][0]][notE[i][1]] = x[notE[i][0]][notE[i][1]] - mu*G[notE[i][0]][notE[i][1]]
  return y

def znew(z, x, notE, mu):
  size = len(notE)
  G = grad(x, mu)
  for i in range(size):
    z[notE[i][0]][notE[i][1]] += -(mu/2)*(k+1)*G[notE[i][0]][notE[i][1]]
  return z

def descen(x, notE, mu, eps):
  z = copy.deepcopy(x)
  y = copy.deepcopy(x)
  k = 0
  size = len(notE)
  f = 0
  while f == 0:
    
    y = ynew(y, x, notE, mu)
    z = znew(z, x, notE, mu)
    x_1 = z*(2/(k+3)) + y*((k+1)/(k+3))

    k += 1
    print(np.linalg.norm(x_1 - x, ord='nuc'))
    if np.linalg.norm(x_1 - x, ord = 'nuc') < eps:
      f = 1
    x = x_1

  return x








In [ ]:




def descent(x, notE, mu, eps):
  y = copy.deepcopy(x)
  z = copy.deepcopy(x)
  k = 0
  size = len(notE)
  f = 0
  while f == 0:
    u, s, v = np.linalg.svd(x, full_matrices = False)
    
    s1 = np.minimum(np.ones_like(s), s/mu)
    #print(u, s, v, s1)
    G = np.dot(u*s1, v)
    #print(G)
    for i in range(size):
      y[notE[i][0]][notE[i][1]] = x[notE[i][0]][notE[i][1]] - mu*G[notE[i][0]][notE[i][1]]
      z[notE[i][0]][notE[i][1]] += -(mu/4)*(k+1)*G[notE[i][0]][notE[i][1]]
    x_1 = z*(2/(k+3)) + y*((k+1)/(k+3))

    k += 1
    print(np.linalg.norm(x_1 - x, ord='nuc'))
    if np.linalg.norm(x_1 - x, ord = 'nuc') < eps:
      f = 1
    x = x_1

  return x

In [ ]:
s = 7
M = np.random.randint(1, 100, size=(s, s))
print(M)
si = 50
E = np.random.randint(0, s, size=(si, 2))
#print(E)
X = np.full_like(M, 100)
#print(X)
for i in range(si):
  X[E[i][0], E[i][1]] = M[E[i][0], E[i][1]]

print(X)
notE = []
for i in range(s):
  for j in range(s):
    if exist(E, i, j) == False:
      notE.append([i,j])
#print(E)
print(len(notE))
Res = descent(X, notE, 20, 0.001)
print(Res)
print(M)
print(np.linalg.norm(M - Res, ord = 'nuc'))

Выходные данные были обрезаны до нескольких последних строк (5000).
0.15364907562953323
0.13575015781287186
0.10604222422260821
0.09232686145566775
0.06923563298930015
0.051124209871589825
0.0594432730175909
0.046363446483340585
0.04474931012871546
0.04524407492580346
0.045447965545123564
0.0457687055886235
0.03993839772717998
0.04786420982180634
0.04209641148174219
0.037823468551056436
0.03490872550473779
0.0364726963086081
0.030639239969931252
0.031501908984392264
0.03010838529415274
0.041120180861818775
0.033052773217391224
0.04251547308171718
0.029330365107866487
0.04075998143940636
0.030451507615450027
0.032566182637876064
0.03963504229868996
0.03798199749189636
0.033695133043948305
0.035103989526613776
0.032921326971224484
0.029414263257139497
0.028917002542253344
0.033615948369052476
0.027627880418524015
0.02807227246002067
0.026729921409778708
0.050346812551774034
0.02964511541194616
0.030741547141192902
0.031068485975661142
0.031114217223303077
0.03220788031213247
0.0360303143

In [ ]:
from PIL import Image
im_1 = Image.open(r"1.jpg")
pic = np.array(im_1)
#print(pic)
Ox = len(pic)
Oy = len(pic[0])
picR = np.zeros((Ox, Oy))
picG = np.zeros((Ox, Oy))
picB = np.zeros((Ox, Oy))
for i in range(Ox):
  for j in range(Oy):
    picR[i,j] = pic[i, j, 0]
    picG[i,j] = pic[i, j, 1]
    picB[i,j] = pic[i, j, 2]
print(Ox, Oy)


400 620


In [ ]:
import random

def delet(Ox, Oy, perc, n = 1):
  m = int(Ox*Oy//(100/perc))
  if n == 1:
    deleted = []
    pool = np.random.choice(Ox*Oy, m, replace=False)
    for k in range(m):
      deleted.append([pool[k]//Oy, pool[k]%Oy])
    return deleted
  elif n == 2:
    deleted = []
    pool = np.random.choice(Ox, m//Oy, replace = False)
    for k in range(len(pool)):
      for i in range(Oy):
        deleted.append([pool[k], i])
    return deleted
  elif n == 3:
    deleted = []
    pool = np.random.choice(Oy, m//Ox, replace = False)
    for k in range(len(pool)):
      for i in range(Ox):
        deleted.append([i, pool[k]])
    return deleted
  elif n == 4:
    deleted = []
    pool1 = np.random.choice(Ox, m//(Oy*2) + 1, replace = False)
    pool2 = np.random.choice(Oy, m//(Ox*2) + 1, replace = False)
    for k in range(len(pool1)):
      for i in range(Oy):
        deleted.append([pool1[k], i])
    for k in range(len(pool2)):
      for i in range(Ox):
        deleted.append([i, pool2[k]])
    return deleted
  elif n == 5:
    deleted = []
    
    dlina = np.random.randint(m//Oy + 1, Ox)
    print(dlina)
    height = (m)//(dlina) + 1
    print(height)
    n1 = np.random.choice(Ox-dlina)
    m1 = np.random.choice(Oy - height)
    for i in range(dlina):
      for j in range(height):
        deleted.append([n1 + i, m1 + j])
    return deleted

In [ ]:
import random
#n = random.sample(range(1, 100), 10)
#print(n)
per = 25
m = int(Ox*Oy//(100/per))
#print(m)
#pool = np.random.choice(Ox*Oy, m, replace=False)
#deleted = []
#for k in range(m):
#  deleted.append([pool[k]//Oy, pool[k]%Oy])
#print(deleted)
deleted = delet(Ox, Oy, per, 5)
XR = copy.deepcopy(picR)
XG = copy.deepcopy(picG)
XB = copy.deepcopy(picB)
for k in range(m):
  XR[deleted[k][0], deleted[k][1]] = 256
  XG[deleted[k][0], deleted[k][1]] = 256
  XB[deleted[k][0], deleted[k][1]] = 256
X = np.full_like(pic, 0)
for i in range(Ox):
  for j in range(Oy):
    X[i, j, 0] = XR[i,j]
    X[i, j, 1] = XG[i,j]
    X[i, j, 2] = XB[i,j]
img = Image.fromarray(X, 'RGB')
img.save('broken.png')


272
228


In [ ]:
#print(XR)
RedRes = descent(XR, deleted, 400, 0.1)
print(RedRes)

1247.151245662194
1195.840080104595
1102.7800785825207
1034.219177382873
994.6340726513918
970.7506224113855
956.3175725618676
948.1947379231326
944.7726306647488
944.881952561648
947.6565513296816
952.7057131735658
959.9238371350874
969.438233838327
981.3160081014985
995.2694388082369
1010.912608543302
1027.56206367087
1043.8163149831296
1057.8541149259809
1067.9450763468328
1072.822825504855
1071.7953878452204
1064.5067728578533
1051.071676933583
1032.0650872626998
1008.2275496549811
980.2000616429768
948.5883436199746
914.063642740155
877.323350972859
839.0363374294463
799.8458694561706
760.3878782939049
721.2673085662316
683.0208613117503
646.1160947991586
610.9246311028976
577.6714888007309
546.4001924031651
516.9934983827364
489.24413478216024
462.93662802977957
437.9131111805693
414.10438702980593
391.5057930008206
370.1028339602779
349.82969337590674
330.60697531945243
312.3805956107959
295.12291050117534
278.82083499922214
263.4595274218661
249.00530278590355
235.4064488804340

In [ ]:
GreenRes = descent(XG, deleted, 400, 0.1)


1301.254949619939
1251.6794928861268
1152.3793147174654
1082.0668519986916
1038.6037279462469
1010.1900172988388
990.9901361921222
979.26533211686
973.1705686904274
971.5821422984803
973.8211366512411
979.4325284650109
987.9886975229228
999.1776178187648
1012.3703665364214
1026.7398130405747
1041.852446543063
1057.6092300178493
1074.108637761436
1090.8626075100801
1106.5160524550656
1119.2690834742696
1126.9078247962982
1127.2501853837857
1118.9361164443894
1101.7004659338836
1076.0528652816758
1042.663604109915
1002.7672530297591
958.3300514771078
911.4479646171446
863.789269776195
816.4430747093736
769.9471440805374
724.3285860462901
679.5690100228135
635.752774384442
593.089833638993
551.9023324525854
512.6069497612997
475.66297489463255
441.47559194371786
410.29219271515586
382.1445325330726
356.8818601510597
334.24685633208946
313.8850544071584
295.3151209587816
278.027386994808
261.58968382717774
245.68889175923337
230.12870766678236
214.80975399551943
199.70712663832268
184.8495

In [ ]:
BlueRes = descent(XB, deleted, 400, 0.1)

1466.1635982051948
1394.762514185804
1293.6986273520822
1208.2981476080206
1142.9936665127289
1091.5194650357446
1050.6589408135032
1019.5285689405434
997.5732503768971
983.7200173931295
976.972004459424
976.5833817621275
981.8369235751395
992.0983978088017
1006.4486179467398
1023.8742750896114
1043.8338908031774
1065.9890434860447
1090.1519374173695
1116.1242653147535
1142.8289370851824
1167.6289094838073
1187.8918487111316
1201.9477682447462
1208.7003258178836
1207.3169236827355
1196.9553222904406
1176.8148444098085
1146.705442825509
1107.4878808155554
1061.0269934634796
1009.7734091875335
956.2213326481982
902.4463626577547
849.8260738681643
799.1123055232696
750.4711669270843
703.7025981385325
658.5217124788524
614.7389720357153
572.308352730538
531.2917582059787
491.80513445917364
453.9782151512137
417.93359586807986
383.77655836974515
351.5692118217458
321.2955646283772
292.8747146034282
266.2114096580987
241.22287855850615
217.8457844452447
196.04223085927015
175.8076233891286
1

In [ ]:
R = np.full_like(pic, 0)
for i in range(Ox):
  for j in range(Oy):
    R[i, j, 0] = RedRes[i,j]
    R[i, j, 1] = GreenRes[i,j]
    R[i, j, 2] = BlueRes[i,j]
img = Image.fromarray(R, 'RGB')
img.save('rebild5.png')